In [1]:
!pip --version


pip 24.2 from C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip (python 3.12)



In [2]:
!pip install selenium beautifulsoup4
!pip install pandas
!pip install requests


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


!pip install XlsxWriter


In [3]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


In [4]:

# Hàm kiểm tra kết nối internet
def check_internet():
    # URL trang web để kiểm tra kết nối, thường dùng Google vì nó ổn định
    url = 'http://www.google.com'
    # Thiết lập thời gian chờ (timeout) là 5 giây
    timeout = 5
    try:
        # Gửi yêu cầu GET đến Google với thời gian chờ 5 giây
        requests.get(url, timeout=timeout)
        # Nếu không có lỗi, kết nối internet đang hoạt động
        return True
    # Bắt ngoại lệ nếu có lỗi kết nối hoặc vượt quá thời gian chờ
    except (requests.ConnectionError, requests.Timeout):
        # Nếu có lỗi kết nối hoặc timeout, hàm sẽ trả về False
        return False

# Hàm lưu dữ liệu vào file
def save_links(data, file_name="link_collected.txt"):
    # Mở file trong chế độ "append" (thêm nội dung vào cuối file)
    with open(file_name, "a") as file:
        # Lặp qua từng mục (item) trong dữ liệu (data)
        for item in data:
            # Ghi từng mục vào file dưới dạng chuỗi và thêm xuống dòng
            file.write(f"{item}\n")



In [30]:
# Tạo đối tượng service để chỉ định đường dẫn tới file chromedriver
service = Service(executable_path="chromedriver.exe")
# Tạo driver Chrome với đối tượng service vừa khởi tạo
driver = webdriver.Chrome(service=service)

# Kiểm tra kết nối trước khi tiếp tục
if not check_internet():  # Nếu không có kết nối internet
    print("Không có kết nối internet. Đang chờ...")
    while not check_internet():  # Chờ cho đến khi có kết nối trở lại
        time.sleep(5)  # Kiểm tra lại sau mỗi 5 giây
    print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")

# Mở trang Google
try:
    driver.get("https://google.com")
except Exception as e:
    # Nếu xảy ra lỗi (ví dụ: mất kết nối) khi mở Google
    print(f"Lỗi khi mở trang Google: {e}")
    while not check_internet():  # Chờ cho đến khi có kết nối trở lại
        print("Mất kết nối internet. Đang chờ...")
        time.sleep(5)
    print("Đã khôi phục kết nối internet. Tiếp tục...")
    driver.get("https://google.com")  # Mở lại Google khi có kết nối

# Chờ thanh tìm kiếm xuất hiện
try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, "gLFyf"))
    )
except Exception as e:
    # Nếu xảy ra lỗi khi chờ đợi thanh tìm kiếm
    print(f"Lỗi khi chờ thanh tìm kiếm: {e}")
    while not check_internet():  # Chờ cho đến khi có kết nối trở lại
        print("Mất kết nối internet. Đang chờ...")
        time.sleep(5)
    print("Đã khôi phục kết nối internet. Tiếp tục...")

# Tương tác với thanh tìm kiếm
input_element = driver.find_element(By.CLASS_NAME, "gLFyf")  # Tìm thanh tìm kiếm bằng class name
input_element.send_keys("nguyenkim" + Keys.ENTER)  # Nhập từ khóa "nguyenkim" và nhấn Enter

# Xử lý CAPTCHA thủ công nếu có
# print("Chờ bạn xử lý CAPTCHA...")
# time.sleep(30)  # Dừng chương trình trong 30 giây để người dùng xử lý CAPTCHA
# print("Tiếp tục sau khi CAPTCHA đã được xử lý.")

# Chờ kết quả tìm kiếm xuất hiện và tìm liên kết của trang Nguyễn Kim
try:
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng"))
    )
except Exception as e:
    # Nếu xảy ra lỗi khi chờ kết quả tìm kiếm
    print(f"Lỗi khi chờ kết quả tìm kiếm: {e}")
    while not check_internet():  # Chờ cho đến khi có kết nối trở lại
        print("Mất kết nối internet. Đang chờ...")
        time.sleep(5)
    print("Đã khôi phục kết nối internet. Tiếp tục...")

# Nhấp vào liên kết trang Nguyễn Kim
try:
    click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
    click_result.click()
except Exception as e:
    # Nếu xảy ra lỗi khi nhấp vào liên kết
    print(f"Lỗi khi nhấp vào liên kết: {e}")
    while not check_internet():  # Chờ cho đến khi có kết nối trở lại
        print("Mất kết nối internet. Đang chờ...")
        time.sleep(5)
    print("Đã khôi phục kết nối internet. Tiếp tục...")
    click_result = driver.find_element(By.PARTIAL_LINK_TEXT, "Nguyễn Kim - Mua tủ lạnh, máy lạnh, tivi, máy giặt chính hãng")
    click_result.click()  # Nhấp vào liên kết khi có kết nối

# Chờ thêm 10 giây để trang sản phẩm tải xong
time.sleep(10)


In [31]:
# Tạo danh sách lưu trữ các liên kết đã thu thập từ file (nếu có)
saved_links = []

# Khởi tạo biến link1 để chứa danh sách liên kết với bản sao của saved_links
link1 = saved_links.copy()  # Tạo bản sao để tránh sửa đổi danh sách gốc

# Tìm phần tử có class là 'nk-product-cate-homepage' và cuộn đến đó
scroll = driver.find_element(By.CLASS_NAME, 'nk-product-cate-homepage')
driver.execute_script("arguments[0].scrollIntoView();", scroll)  # Cuộn trang đến phần tử được tìm thấy
time.sleep(3)  # Tạm dừng 3 giây để đảm bảo trang tải xong sau khi cuộn

# Kiểm tra kết nối internet trước khi tiếp tục thu thập dữ liệu
if not check_internet():
    print("Không có kết nối internet. Đang chờ...")
    while not check_internet():  # Nếu mất kết nối, chờ cho đến khi khôi phục
        time.sleep(5)  # Kiểm tra lại sau mỗi 5 giây
    print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")

# Tìm các phần tử chứa danh mục sản phẩm có class 'nk-content'
cl_tags = driver.find_elements(By.CLASS_NAME, 'nk-content')

# Lặp qua từng phần tử chứa danh mục sản phẩm
for tag in cl_tags:
    try:
        # Kiểm tra kết nối trong suốt quá trình thu thập
        if not check_internet():
            print("Không có kết nối internet. Đang dừng...")
            while not check_internet():  # Chờ đến khi có kết nối lại
                print("Chờ khôi phục kết nối...")
                time.sleep(5)
            print("Đã khôi phục kết nối. Tiếp tục thu thập dữ liệu...")

        # Lấy tất cả các phần tử <a> trong mỗi thẻ danh mục sản phẩm
        a_tags = tag.find_elements(By.TAG_NAME, 'a')
        for a_tag in a_tags:
            # Lấy thuộc tính 'href' của thẻ <a> để lấy đường dẫn liên kết
            href = a_tag.get_attribute('href')
            # Kiểm tra nếu link tồn tại và chưa có trong link1, thêm link vào danh sách
            if href and href not in link1:
                link1.append(href)  # Thêm link vào danh sách
                print(f"Thu thập link: {href}")
                save_links([href])  # Lưu link vào file ngay lập tức để tránh mất dữ liệu nếu ngắt kết nối
    except Exception as e:
        # Xử lý nếu có lỗi trong quá trình thu thập
        print(f"Đã xảy ra lỗi: {e}")

# Thông báo số lượng liên kết đã thu thập và lưu trữ
print(f"Đã thu thập và lưu {len(link1)} liên kết.")


Thu thập link: https://www.nguyenkim.com/may-lanh/
Thu thập link: https://www.nguyenkim.com/dien-thoai-di-dong/
Thu thập link: https://www.nguyenkim.com/tu-lanh/
Thu thập link: https://www.nguyenkim.com/tivi/
Thu thập link: https://www.nguyenkim.com/may-tinh-xach-tay/
Thu thập link: https://www.nguyenkim.com/may-giat/
Thu thập link: https://www.nguyenkim.com/quat-vi/?features_hash=65-148987-148986-5954
Thu thập link: https://www.nguyenkim.com/may-nuoc-nong/
Thu thập link: https://www.nguyenkim.com/may-in-van-phong/
Thu thập link: https://www.nguyenkim.com/noi-com-dien/
Thu thập link: https://www.nguyenkim.com/noi-chien-khong-dau/
Thu thập link: https://www.nguyenkim.com/may-loc-khong-khi/
Thu thập link: https://www.nguyenkim.com/man-hinh-lcd-vi-tinh/
Thu thập link: https://www.nguyenkim.com/may-ep-trai-cay/
Thu thập link: https://www.nguyenkim.com/may-loc-nuoc/
Thu thập link: https://www.nguyenkim.com/quat-vi/
Thu thập link: https://www.nguyenkim.com/bep-tu-hong-ngoai
Thu thập link: ht

In [8]:
# Hàm lưu dữ liệu vào file
def save_data(data, file_name="data_products.txt"):
    # Mở file ở chế độ "a" (append - nối thêm vào cuối file) để ghi dữ liệu
    with open(file_name, "a") as file:
        # Lặp qua từng mục (item) trong danh sách data
        for item in data:
            # Ghi từng mục vào file và xuống dòng sau mỗi mục
            file.write(f"{item}\n")


In [9]:
product_links = []  # Tạo một danh sách trống để lưu liên kết các sản phẩm

# Duyệt qua từng link danh mục sản phẩm trong danh sách link1
for link in link1:
    is_last_page = False  # Cờ kiểm tra xem đã đến trang cuối cùng của danh mục chưa
    
    while True:
        # Kiểm tra kết nối internet trước khi truy cập link
        if not check_internet():
            print("Không có kết nối internet. Đang chờ...")
            while not check_internet():  # Chờ đến khi có kết nối lại
                time.sleep(5)
            print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")

        try:
            # Mở link danh mục sản phẩm trong trình duyệt
            driver.get(link)

            # Vòng lặp để duyệt qua tất cả các trang trong danh mục sản phẩm
            while not is_last_page:
                try:
                    # Chờ cho đến khi phần tử chứa sản phẩm được tải
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid')]"))
                    )

                    # Lấy tất cả container sản phẩm
                    product_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'nk-product-cate-style-grid') and contains(@class, 'nk-product-collection')]")
                    
                    for container in product_containers:
                        # Kiểm tra lại kết nối internet trong quá trình thu thập dữ liệu
                        if not check_internet():
                            print("Không có kết nối internet. Đang chờ...")
                            while not check_internet():  # Chờ khôi phục kết nối
                                time.sleep(5)
                            print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")

                        try:
                            # Lấy tất cả thẻ <a> chứa link sản phẩm trong container
                            a_tags = container.find_elements(By.TAG_NAME, 'a')
                            for a_tag in a_tags:
                                href = a_tag.get_attribute('href')
                                # Thêm link vào danh sách nếu chưa tồn tại trong product_links
                                if href and href not in product_links:
                                    product_links.append(href)  # Lưu link vào danh sách
                                    save_data([href])  # Ghi link vào file
                                    print(f"Link thu được: {href}")
                        except Exception as e:
                            print(f"Lỗi khi lấy link từ container sản phẩm: {e}")

                    # Kiểm tra nút chuyển trang tiếp theo
                    try:
                        next_page_button = driver.find_element(By.XPATH, "//div[contains(@class, 'NkReview_footer_col-3')]")
                        # Cuộn đến vị trí của nút
                        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                        time.sleep(2)

                        # Kiểm tra nếu đây là trang cuối cùng
                        if not next_page_button.is_displayed():
                            is_last_page = True  # Đặt cờ kết thúc trang
                        else:
                            # Lấy link của trang tiếp theo và truy cập
                            a_tag = next_page_button.find_element(By.TAG_NAME, 'a')
                            href = a_tag.get_attribute('href')
                            driver.get(href)
                            time.sleep(3)
                    except:
                        print("Không tìm thấy trang tiếp theo. Chuyển sang danh mục khác.")
                        is_last_page = True  # Đặt cờ kết thúc danh mục hiện tại

                except Exception as e:
                    print(f"Lỗi thu thập: {e}")
                    is_last_page = True  # Kết thúc nếu có lỗi trong việc tải trang
                    break

            # Kết thúc danh mục sản phẩm hiện tại
            print("Đã xử lý xong danh mục. Chuyển sang danh mục tiếp theo.")
            break

        except Exception as e:
            print(f"Lỗi khi truy cập link: {e}")
            break  # Kết thúc danh mục hiện tại khi có lỗi truy cập


Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=135768
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=132415
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=136668
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=134687
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=132418
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=135713
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=127685
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=102494
Đã xử lý xong danh mục. Chuyển sang danh mục tiếp theo.
Link thu được: https://www.nguyenkim.com/index.php?dispatch=product_features.add_product

In [10]:
print("Tổng sản phẩm: ",len(product_links))

Tổng sản phẩm:  4379


In [11]:
# Lọc link lỗi 404
filtered_links = [link for link in product_links if not link.startswith("https://www.nguyenkim.com/index.php?dispatch=product_features.add_product&product_id=")]

In [12]:
print("Tổng số link: ",len(filtered_links))

Tổng số link:  2204


In [13]:
filtered_links

['https://www.nguyenkim.com/may-lanh-casper-inverter-tc-09is35.html',
 'https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-pu12zkh-8m.html',
 'https://www.nguyenkim.com/may-lanh-reetech-1-hp-rt9-rc9-ta-bt.html',
 'https://www.nguyenkim.com/may-lanh-panasonic-inverter-cu-cs-xu9zkh-8.html',
 'https://www.nguyenkim.com/may-lanh-daikin-ftky25wmvmv.html',
 'https://www.nguyenkim.com/may-lanh-casper-inverter-gc-09is35.html',
 'https://www.nguyenkim.com/may-lanh-mitsubishi-inverter-msy-muy-jw25vf.html',
 'https://www.nguyenkim.com/may-lanh-casper-inverter-gc-12is35.html',
 'https://www.nguyenkim.com/may-lanh-sharp-inverter-ah-x10zew.html',
 'https://www.nguyenkim.com/may-lanh-toshiba-inverter-1-hp-ras-h10e2kcvg-v.html',
 'https://www.nguyenkim.com/may-lanh-aqua-inverter-aqa-rv13qc.html',
 'https://www.nguyenkim.com/may-lanh-daikin-ftky35wmvmv.html',
 'https://www.nguyenkim.com/may-lanh-daikin-inverter-1-hp-ftkz25vvmv.html',
 'https://www.nguyenkim.com/may-lanh-sharp-inverter-ah-x13ze

In [27]:
data = []  # Tạo danh sách trống để lưu thông tin từng sản phẩm

# Duyệt qua các liên kết sản phẩm trong filtered_links từ vị trí 1000 đến 1001
for op_link in filtered_links[1000:1001]:
    while True:
        # Kiểm tra kết nối internet trước khi truy cập trang
        if not check_internet():
            print("Không có kết nối internet. Đang chờ...")
            while not check_internet():  # Chờ kết nối lại
                time.sleep(5)
            print("Đã khôi phục kết nối internet. Tiếp tục quá trình...")
        
        try:
            # Mở liên kết sản phẩm
            driver.get(op_link)
            time.sleep(1)

            # Lấy tên sản phẩm
            try:
                name = driver.find_element(By.CLASS_NAME, "wrap_name_vote").find_element(By.TAG_NAME, 'h1').text
            except:
                name = "Sản phẩm không có tên"  # Gán giá trị mặc định nếu không tìm thấy tên
            
            # Lấy giá khuyến mãi của sản phẩm
            try:
                value_final = driver.find_element(By.CLASS_NAME, "product_info_price_value-final").find_element(By.TAG_NAME, 'span').text
            except:
                value_final = ""  # Gán giá trị rỗng nếu không tìm thấy giá khuyến mãi
            
            # Lấy giá ban đầu của sản phẩm
            try:
                value_real = driver.find_element(By.CLASS_NAME, "price_promotion").find_element(By.CLASS_NAME, "product_info_price_value-real").text
            except:
                value_real = value_final  # Nếu không có giá ban đầu, gán bằng giá khuyến mãi

            # Lấy thông tin chi tiết sản phẩm
            try:
                div_specification = driver.find_element(By.CLASS_NAME, "productSpecification_table")
                # Cuộn đến phần tử chứa thông tin chi tiết
                ActionChains(driver).move_to_element(div_specification).perform()
                time.sleep(2)

                # Lấy mã HTML của phần tử để phân tích bằng BeautifulSoup
                html_content = div_specification.get_attribute('outerHTML')
                print(html_content)
                soup = BeautifulSoup(html_content, 'html.parser')
                t_body = soup.find('tbody')
                rows = t_body.find_all('tr')[:5]  # Lấy tối đa 5 hàng thông tin

                # Xử lý dữ liệu chi tiết sản phẩm
                product_details = {}
                for i, row in enumerate(rows):
                    title = row.find('td', class_='title').text.strip()  # Lấy tiêu đề của thuộc tính
                    value = row.find('td', class_='value').text.strip()  # Lấy giá trị của thuộc tính
                    product_details[title] = value

                # Đảm bảo các trường thông tin cơ bản có mặt
                model = product_details.get("Model:", "")
                color = product_details.get("Màu sắc:", "")
                manufacturer = product_details.get("Nhà sản xuất:", "")
                origin = product_details.get("Xuất xứ:", "")
                release_year = product_details.get("Năm ra mắt :", "")
                
            except Exception as e:
                # Nếu có lỗi, đặt các trường thông tin về rỗng
                print(f"Lỗi khi lấy thông tin sản phẩm: {e}")
                model = color = manufacturer = origin = release_year = ""

            # Thêm thông tin sản phẩm vào danh sách
            data.append({
                'Tên sản phẩm': name,
                'Giá khuyến mãi': value_final,
                'Giá ban đầu': value_real,
                'Model': model,
                'Màu sắc': color,
                'Nhà sản xuất': manufacturer,
                'Xuất xứ': origin,
                'Năm ra mắt': release_year
            })
        
        except Exception as e:
            # Bắt và in thông báo lỗi nếu không truy cập được link sản phẩm
            print(f"Lỗi: {e}")
            break
        
        break  # Thoát vòng lặp khi xử lý xong sản phẩm

# Tạo DataFrame từ danh sách dữ liệu sản phẩm
data_df = pd.DataFrame(data)


<table class="productSpecification_table"><caption class="productSpecification_title">Thông số kỹ thuật</caption><tbody><tr><td class="title">Model:</td><td class="value"> TV2725SV9J </td></tr><tr><td class="title">Màu sắc:</td><td class="value"> Đen </td></tr><tr><td class="title">Nhà sản xuất:</td><td class="value"> LG </td></tr><tr><td class="title">Xuất xứ:</td><td class="value"> Thái Lan </td></tr><tr><td class="title">Năm ra mắt :</td><td class="value"> 2024 </td></tr><tr><td class="title">Thời gian bảo hành:</td><td class="value"> 24 tháng </td></tr><tr><td class="title">Địa điểm bảo hành:</td><td class="value"> Nguyễn Kim </td></tr><tr><td class="title">Bảng điều khiển:</td><td class="value"> Cảm ứng, ngôn ngữ Tiếng Việt/Tiếng Anh </td></tr><tr><td class="title">Loại máy giặt:</td><td class="value"> Cửa trên </td></tr><tr><td class="title">Khối lượng giặt:</td><td class="value"> 25 kg </td></tr></tbody></table>
                            Tên sản phẩm Giá khuyến mãi  Giá ban đầ

In [28]:
data_df

,Tên sản phẩm,Giá khuyến mãi,Giá ban đầu,Model,Màu sắc,Nhà sản xuất,Xuất xứ,Năm ra mắt
0,Máy giặt LG Inverter 25 kg TV2725SV9J,18.490.000đ,22.990.000đ,TV2725SV9J,Đen,LG,Thái Lan,2024


In [18]:
data_df.to_excel("output.xlsx", index=False)